In [1]:
import pandas as pd
import os
import matplotlib
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

joined = None

def load_raw_data():
    global airports, airport_frequencies, joined
    airports = pd.read_csv('airports.csv')
    airport_frequencies = pd.read_csv('airport-frequencies.csv')
    joined = airports.set_index('ident').join(airport_frequencies.set_index('airport_ident'), rsuffix='_right')

In [2]:
def get_average_text():
    if joined is None:
        tk.messagebox.showwarning(message='No data has been loaded.')
        return
    
    result = ''
    
    # 3Ai - Generate mean for frequency_mhz for large airports
    large_airports = joined[joined['type'] == 'large_airport']
    result += f"Large airport mean frequency: {large_airports['frequency_mhz'].mean():.2f}\n"
    
    # Generate median for fmhz for large airports
    result += f"Large airport median frequency: {large_airports['frequency_mhz'].median():.2f}\n"
    
    # Generate mode for fmhz for large airports
    for value in large_airports['frequency_mhz'].mode():
        result += f"Large airport mode frequency: {value:.2f}\n"
    
    # 3Aii - Generate mean for frequency_mhz for frequencies over 100 mhz
    over_100mhz = joined[joined['frequency_mhz'] > 100]
    result += f"Mean frequency over 100 mhz: {over_100mhz['frequency_mhz'].mean():.2f}\n"
    result += f"Median frequency over 100 mhz: {over_100mhz['frequency_mhz'].median():.2f}\n"
    for value in over_100mhz['frequency_mhz'].mode():
        result += f"Mode frequency over 100 mhz: {value:.2f}"
    
    return result

In [3]:
def generate_frequency_graph(airport_type):
    fig = Figure(figsize=(4, 4), dpi=100)
    ax = fig.add_subplot(111)
    ax.set_title(airport_type)
    
    airports = joined[joined['type'] == airport_type]
    airports['frequency_mhz'].sort_values().plot(ax=ax)
    
    return FigureCanvasTkAgg(fig, master=window)
    
def show_frequency_graphs():
    if joined is None:
        tk.messagebox.showwarning(message='No data has been loaded.')
        return
    row = 3
    column = 0
    for airport_type in ('small_airport', 'medium_airport', 'large_airport'):
        canvas = generate_frequency_graph(airport_type)
        canvas.get_tk_widget().grid(row=row, column=column, columnspan=4)
        column += 4

In [4]:
#Import tkinter library
import tkinter as tk
import tkinter.ttk as ttk
#Create an instance of Tkinter frame or window
window = tk.Tk()
#Set the geometry of tkinter frame
window.geometry("1600x800")
text_output = ttk.Label(window)

def set_text_output(text):
    text_output['text'] = text
  
def filter_frequencies():
    global joined
    if joined is None:
        tk.messagebox.showwarning(message='No data has been loaded.')
        return
    try:
        min_f = float(min_frequency.get('1.0', 'end'))
        max_f = float(max_frequency.get('1.0', 'end'))
        joined = joined[(joined['frequency_mhz'] >= min_f) & (joined['frequency_mhz'] <= max_f)]
    except ValueError:
        tk.messagebox.showwarning(message='Please enter numbers in the text boxes')

def save_prepared():
    if joined is None:
        tk.messagebox.showwarning(message='No data has been loaded.')
        return
    filename = 'Prepared.xlsx'
    if os.path.exists(filename):
        window.update()
        result = tk.messagebox.askyesno(message='The file already exists, do you want to overwrite it?')
        if result == False:
            return
    joined.to_excel('Prepared.xlsx')

def load_prepared():
    joined = pd.read_excel('Prepared.xlsx')
    
buttons = [
    ttk.Button(window, text='Load raw data', command=load_raw_data),
    ttk.Button(window, text='Load prepared data', command=load_prepared),
    ttk.Button(window, text='Save prepared data', command=save_prepared),
    ttk.Button(window, text='Show averages', command=lambda: set_text_output(get_average_text())),
    ttk.Button(window, text='Show frequency graphs', command=lambda: show_frequency_graphs())
]

row = 0
column = 0

for button in buttons:
    button.grid(row=row, column=column)
    column += 1
    
text_output.grid(row=2, column=0, columnspan=5)

# options for filtering out certain frequencies
min_frequency_label = tk.Label(window, text='Min frequency')
max_frequency_label = tk.Label(window, text='Max frequency')
min_frequency = tk.Text(window, height=1, width=10)
max_frequency = tk.Text(window, height=1, width=10)
min_frequency_label.grid(row=1, column=0)
min_frequency.grid(row=1, column=1)
max_frequency_label.grid(row=1, column=2)
max_frequency.grid(row=1, column=3)
filter_button = ttk.Button(window, text='Filter frequencies', command=filter_frequencies)
filter_button.grid(row=1, column=4)

window.mainloop()